**implementation refrences**:

1. https://colab.research.google.com/drive/1D45E5bUK3gQ40YpZo65ozs7hg5l-eo_U?usp=sharing
2. https://colab.research.google.com/drive/1oO-Raqge8oGXGNkZQOYTH-je4Xi1SFVI?usp=sharing

# Imports and installations

In [ ]:
!pip install torch==1.12.0 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install plotly
!pip install pytorch-metric-learning -q 

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
import torch_geometric.transforms as T
from torch_geometric.datasets import ModelNet
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, PointConv, fps, global_max_pool, radius ,DynamicEdgeConv ,EdgeConv 
import plotly.express as px
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from pytorch_metric_learning.losses import NTXentLoss

from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import random
import gc

In [ ]:
# reproducibility stuff
torch.manual_seed(42)
np.random.seed(42)
random.seed(0)

torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True  # Note that this Deterministic mode can have a performance impact
torch.backends.cudnn.benchmark = False

# Utility Functions


In [ ]:
#train vanilla models
def train_model(model,train_loader):
    model.train()

    total_loss = 0
    for data in tqdm(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()
    return total_loss / len(train_loader)

def evaluate_model(model,val_loader):
    model.eval()
    correct = 0
    total_loss = 0
    with torch.no_grad():
        for data in tqdm(val_loader):
            data = data.to(device)
            out = model(data)
            loss = F.nll_loss(out, data.y)
            total_loss += loss.item() * data.num_graphs
            pred = model(data).max(dim=1)[1]
            correct += pred.eq(data.y).sum().item()
    return (total_loss / len(val_loader), correct / len(val_loader.dataset))


def test_model(model,loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        with torch.no_grad():
            pred = model(data).max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)

#train  contrastive learninig
def train_contrastive(model,data_loader):
    model.train()
    total_loss = 0
    for _, data in enumerate(tqdm(data_loader)):
        data = data.to(device)
        optimizer.zero_grad()
        # Get data representations
        h_1, h_2, compact_h_1, compact_h_2 = model(data)
        # Prepare for loss
        embeddings = torch.cat((compact_h_1, compact_h_2))
        # The same index corresponds to a positive pair
        indices = torch.arange(0, compact_h_1.size(0), device=compact_h_2.device)
        labels = torch.cat((indices, indices))
        loss = loss_func(embeddings, labels)
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()
    return total_loss / len(data_loader)

In [ ]:
# classes:['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor',
# 'night_stand', 'sofa', 'table', 'toilet']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pre_transform, transform = T.NormalizeScale(), T.SamplePoints(2000)
dataset = ModelNet('modelnet/train/', '10', True, transform, pre_transform)
test_dataset = ModelNet('modelnet/test/', '10', False, transform, pre_transform)
print("Number of train Samples: ", len(dataset))
print("Number of test Samples: ", len(test_dataset))
print("Sample: ", dataset)
print("Sample: ", test_dataset)

Extracting modelnet/train/ModelNet10.zip
Processing...
Done!
Extracting modelnet/test/ModelNet10.zip
Processing...


Number of train Samples:  3991
Number of test Samples:  908
Sample:  ModelNet10(3991)
Sample:  ModelNet10(908)


Done!


# Data preprocessing

In [ ]:
#data processing for getting 10 percent ,1 percent or all(80 percent of data) for training data and validation data
percent_data=0.01 #from [0-0.8]--try for 0.01,0.1,0.8
trainData_size=int(len(dataset)*percent_data)
testData_size=int(trainData_size * 0.2 )
#testData_size = trainData_size # for pre-training on 1 percent of label
print("Training Size",trainData_size)
indices = np.arange(len(dataset))

train_indices, test_indices = train_test_split(indices, train_size = trainData_size,test_size=trainData_size, stratify=dataset.data.y)

# Warp into Subsets and DataLoaders
train_subset = torch.utils.data.Subset(dataset, train_indices)
val_subset = torch.utils.data.Subset(dataset, test_indices)

train_loader = DataLoader(train_subset, shuffle=True, batch_size=32)
val_loader = DataLoader(val_subset, shuffle=False, batch_size=32)

# check distribution of data wrt classes
train_targets = []
for target in train_loader:
    train_targets.append(target.y)
train_targets = torch.cat(train_targets)

print(train_targets.unique(return_counts=True))

Training Size 39
(tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), tensor([1, 5, 9, 2, 2, 4, 2, 7, 4, 3]))


In [ ]:
#test dataset
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=32)

**Plot Some Examples**

In [ ]:
def plot_3d_shape(shape):
#     print("Number of data points: ", shape.x.shape[0])
    x = shape.pos[:, 0]
    y = shape.pos[:, 1]
    z = shape.pos[:, 2]
    fig = px.scatter_3d(x=x, y=y, z=z, opacity=0.3)
    fig.show()

# Pick a sample
sample_idx = 55
# plot_3d_shape(train_subset[sample_idx])

# POINTNET 2

Training Pointnet2 uisng all data withoit any pretraining

In [ ]:
class SAModule(torch.nn.Module):
    def __init__(self, ratio, r, nn):
        super().__init__()
        self.ratio = ratio
        self.r = r
        self.conv = PointConv(nn, add_self_loops=True)

    def forward(self, x, pos, batch):
        idx = fps(pos, batch, ratio=self.ratio)
        row, col = radius(pos, pos[idx], self.r, batch, batch[idx],
                          max_num_neighbors=64)
        edge_index = torch.stack([col, row], dim=0)
        x_dst = None if x is None else x[idx]
        x = self.conv((x, x_dst), (pos, pos[idx]), edge_index)
        pos, batch = pos[idx], batch[idx]
        return x, pos, batch


class GlobalSAModule(torch.nn.Module):
    def __init__(self, nn):
        super().__init__()
        self.nn = nn

    def forward(self, x, pos, batch):
        x = self.nn(torch.cat([x, pos], dim=1))
        x = global_max_pool(x, batch)
        pos = pos.new_zeros((x.size(0), 3))
        batch = torch.arange(x.size(0), device=batch.device)
        return x, pos, batch


class PointNet(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # Input channels account for both `pos` and node features.
        self.sa1_module = SAModule(0.5, 0.2, MLP([3, 64, 64, 128]))
        self.sa2_module = SAModule(0.25, 0.4, MLP([128 + 3, 128, 128, 256]))
        self.sa3_module = GlobalSAModule(MLP([256 + 3, 256, 512, 1024]))

        self.mlp = MLP([1024, 512, 256, 16], dropout=0.5, norm=None)

    def forward(self, data):
        sa0_out = (data.x, data.pos, data.batch)
        sa1_out = self.sa1_module(*sa0_out)
        sa2_out = self.sa2_module(*sa1_out)
        sa3_out = self.sa3_module(*sa2_out)
        x, pos, batch = sa3_out
        return self.mlp(x).log_softmax(dim=-1)



In [ ]:
PointNetwork = PointNet().to(device)
optimizer = torch.optim.Adam(PointNetwork.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
for epoch in range(0, 50):
    loss = train_model(PointNetwork,train_loader)
    test_loss,test_acc = evaluate_model(PointNetwork,val_loader)
    print(f'Epoch {epoch:03d}, Train_Loss: {loss:.4f},Test_Loss: {test_loss:.4f}, Test_acc: {test_acc:.4f}')
    scheduler.step()


In [ ]:
# # load saved model
# PointNetwork = PointNet().to(device)
# PointNetwork.load_state_dict(torch.load('/kaggle/working/PointNet_saved_model_final.pth'))
# PointNetwork.eval()

In [ ]:
test_model(PointNetwork,test_loader)

In [ ]:
torch.save(PointNetwork.state_dict(), 'PointNet_saved_model_final.pth')

In [ ]:
del PointNetwork

# Dynamic Graph CNN
Training EdgeConvNet uisng all data withoit any pretraining

In [ ]:
class EdgeConvNet(torch.nn.Module):
    def __init__(self, out_channels, k=20, aggr='max'):
        super().__init__()

        self.conv1 = DynamicEdgeConv(MLP([2 * 3, 64, 64]), k, aggr)
        self.conv2 = DynamicEdgeConv(MLP([2 * 64, 128]), k, aggr)
        self.lin1 = Linear(128 + 64, 1024)

        self.mlp = MLP([1024, 256, out_channels], dropout=0.5, norm=None)

    def forward(self, data):
        pos, batch = data.pos, data.batch
        x1 = self.conv1(pos, batch)
        x2 = self.conv2(x1, batch)
        out = self.lin1(torch.cat([x1, x2], dim=1))
        out = global_max_pool(out, batch)
        out = self.mlp(out)
        return F.log_softmax(out, dim=1)




In [ ]:
EdgeConvNetwork = EdgeConvNet(10, k=20).to(device)
optimizer = torch.optim.Adam(EdgeConvNetwork.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
for epoch in range(0, 50):
    loss = train_model(EdgeConvNetwork,train_loader)
    test_loss,test_acc = evaluate_model(EdgeConvNetwork,val_loader)
    print(f'Epoch {epoch:03d}, Train_Loss: {loss:.4f},Test_Loss: {test_loss:.4f}, Test_acc: {test_acc:.4f}')
    scheduler.step()

In [ ]:
test_model(EdgeConvNetwork,test_loader)

In [ ]:
torch.save(EdgeConvNetwork.state_dict(), 'EdgeConvNet_saved_model_final.pth')

In [ ]:
del EdgeConvNet

# Pre-training using Contrastive approach

In this section we apply **Augmentations** to the graph data and train in unsupervised manner using NT-Xent Loss

In [ ]:
#augmentation set 1
# augmentation = T.Compose(
#     [
#       T.RandomJitter(0.05), 
#       T.RandomShear(0.1),
#       T.RandomRotate(degrees=180),
#       T.RandomScale((0.1,0.9))
#     ]
# )

In [ ]:
#augmentation set 2
augmentation = T.Compose([T.RandomJitter(0.03),
                          T.RandomFlip(1),
                          T.RandomShear(0.2)])

In [ ]:
sample = next(iter(dataset_loader))
# plot_3d_shape(sample[0])


In [ ]:
transformered = augmentation(sample)
# plot_3d_shape(transformered[0])

# Pre-train Dynamic Graph CNN model

In [ ]:
class EdgeConvNet_ContrasriveNet(torch.nn.Module):
    def __init__(self, k=20, aggr='max'):
        super().__init__()
        # Feature extraction
        self.conv1 = DynamicEdgeConv(MLP([2 * 3, 64, 64]), k, aggr)
        self.conv2 = DynamicEdgeConv(MLP([2 * 64,128 , 128]), k, aggr)
        self.conv3 = DynamicEdgeConv(MLP([2 * 128, 256]), k, aggr)
        # Encoder head 
        self.lin1 = Linear(128 + 64 + 256  , 1024)
        # Projection head (See explanation in SimCLRv2)
        self.ph1 = Linear(1024, 1024)
        self.ph2 = Linear(1024, 512)
        self.ph3 = Linear(512 , 64)
        

    def forward(self, data, train=True):
        if train:
            # Get 2 augmentations of the batch
            augm_1 = augmentation(data)
            augm_2 = augmentation(data)

            # Extract properties
            pos_1, batch_1 = augm_1.pos, augm_1.batch
            pos_2, batch_2 = augm_2.pos, augm_2.batch

            # Get representations for first augmented view
            x1 = self.conv1(pos_1, batch_1)
            x2 = self.conv2(x1, batch_1)
            x3 = self.conv3(x2, batch_1)
            h_points_1 = self.lin1(torch.cat([x1, x2, x3], dim=1))

            # Get representations for second augmented view
            x1 = self.conv1(pos_2, batch_2)
            x2 = self.conv2(x1, batch_2)
            x3 = self.conv3(x2, batch_2)
            h_points_2 = self.lin1(torch.cat([x1, x2, x3], dim=1))
            
            # Global representation
            h_1 = global_max_pool(h_points_1, batch_1)
            h_2 = global_max_pool(h_points_2, batch_2)
        else:
            x1 = self.conv1(data.pos, data.batch)
            x2 = self.conv2(x1, data.batch)
            x3 = self.conv3(x2, data.batch)
            h_points = self.lin1(torch.cat([x1, x2, x3], dim=1))
            return global_max_pool(h_points, data.batch)

        # Transformation for loss function
        c1_h1= self.ph1(h_1)
        c2_h1= F.relu(self.ph2(c1_h1))
        compact_h_1= self.ph3(c2_h1)
        
        c1_h2= self.ph1(h_2)
        c2_h2= F.relu(self.ph2(c1_h2))
        compact_h_2 = self.ph3(c2_h2)
        
        return h_1, h_2, compact_h_1, compact_h_2

In [ ]:
EdgeConvNet_Contrasrive = EdgeConvNet_ContrasriveNet().to(device)
optimizer = torch.optim.Adam(EdgeConvNet_Contrasrive.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
loss_func = NTXentLoss(temperature=0.10)

In [ ]:
for epoch in range(0, 15):
    loss = train_contrastive(EdgeConvNet_Contrasrive,dataset_loader)
    print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')
    scheduler.step()

In [ ]:
torch.save(EdgeConvNet_Contrasrive.state_dict( ), 'EdgeConvNet_Contrasrive_saved_model_final_aug2.pth')

In [ ]:
del EdgeConvNet_Contrasrive

# Pre-train PointNet2 model

In [ ]:
class PointNetContrasriveNet(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # Input channels account for both `pos` and node features.
        self.sa1_module = SAModule(0.5, 0.2, MLP([3, 64, 64, 128]))
        self.sa2_module = SAModule(0.25, 0.3, MLP([128 + 3, 128, 128, 256]))
        self.sa3_module = SAModule(0.25, 0.4, MLP([256 + 3, 256, 256, 512]))
        self.sa4_module = GlobalSAModule(MLP([512 + 3, 512, 512, 1024]))
        # Projection head (See explanation in SimCLRv2)
        self.ph1 = Linear(1024, 1024)
        self.ph2 = Linear(1024, 512)
        self.ph3 = Linear(512 , 64)
        

    def forward(self, data, train=True):
        if train:
            # Get 2 augmentations of the batch
            augm_1 = augmentation(data)
            augm_2 = augmentation(data)
            
            #extraxt features from aug1
            sa0_out = (augm_1.x, augm_1.pos, augm_1.batch)
            sa1_out = self.sa1_module(*sa0_out)
            sa2_out = self.sa2_module(*sa1_out)
            sa3_out = self.sa3_module(*sa2_out)
            sa4_out = self.sa4_module(*sa3_out)
            h_1, _ , _ = sa4_out
            
            #extraxt features from aug1
            sa0_out = (augm_2.x, augm_2.pos, augm_2.batch)
            sa1_out = self.sa1_module(*sa0_out)
            sa2_out = self.sa2_module(*sa1_out)
            sa3_out = self.sa3_module(*sa2_out)
            sa4_out = self.sa4_module(*sa3_out)
            h_2, _, _ = sa4_out
            
        else:
            sa0_out = (data.x, data.pos, data.batch)
            sa1_out = self.sa1_module(*sa0_out)
            sa2_out = self.sa2_module(*sa1_out)
            sa3_out = self.sa3_module(*sa2_out)
            sa4_out = self.sa4_module(*sa3_out)
            h_points, _ , _ = sa4_out
            return h_points

        # Transformation for loss function
        c1_h1= self.ph1(h_1)
        c2_h1= F.relu(self.ph2(c1_h1))
        compact_h_1= self.ph3(c2_h1)
        
        c1_h2= self.ph1(h_2)
        c2_h2= F.relu(self.ph2(c1_h2))
        compact_h_2 = self.ph3(c2_h2)
        
        return h_1, h_2, compact_h_1, compact_h_2

In [ ]:
PointNetContrasrive = PointNetContrasriveNet().to(device)
optimizer = torch.optim.Adam(PointNetContrasrive.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
loss_func = NTXentLoss(temperature=0.10)

In [ ]:
for epoch in range(0, 30):
    loss = train_contrastive(PointNetContrasrive,dataset_loader)
    print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')
    scheduler.step()

In [ ]:
torch.save(PointNetContrasrive.state_dict(),'PointNetContrasrive_test_final30_aug2.pth')

In [ ]:
del PointNetContrasrive

# Plot and retrive representations from pre-trained models

In [ ]:
# Get sample batch
sample = next(iter(dataset_loader))

# Get representations
h = PointNetContrasrive(sample.to(device), train=False)
h = h.cpu().detach()
labels = sample.y.cpu().detach().numpy()

# Get low-dimensional t-SNE Embeddings
h_embedded = TSNE(n_components=2, learning_rate='auto',
                   init='random').fit_transform(h.numpy())

# Plot
ax = sns.scatterplot(x=h_embedded[:,0], y=h_embedded[:,1], hue=labels, 
                    alpha=0.5, palette="tab20")

# Add labels to be able to identify the data points
annotations = list(range(len(h_embedded[:,0])))

def label_points(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.02, point['y'], str(int(point['val'])))

label_points(pd.Series(h_embedded[:,0]), 
            pd.Series(h_embedded[:,1]), 
            pd.Series(annotations), 
            plt.gca()) 

In [ ]:
import numpy as np

def sim_matrix(a, b, eps=1e-8):
    """
    Eps for numerical stability
    """
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
    sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))
    return sim_mt

similarity = sim_matrix(h, h)
max_indices = torch.topk(similarity, k=2)[1][:, 1]
max_vals  = torch.topk(similarity, k=2)[0][:, 1]

# Select index
idx = 3
similar_idx = max_indices[idx]
print(f"Most similar data point in the embedding space for {idx} is {similar_idx}")

Most similar data point in the embedding space for 3 is 16


In [ ]:
plot_3d_shape(sample[idx].cpu())

In [ ]:
plot_3d_shape(sample[similar_idx].cpu())

# Fine_tune on Classification task


**1.Fine tuning Pointnet Network**

In [ ]:
pointnet_path='/kaggle/input/trained-models/PointNetContrasrive_test_final30.pth'
pointnet2_saved=PointNetContrasriveNet().to(device)
pointnet2_saved.load_state_dict(torch.load(pointnet_path))
pointnet2_saved.ph3 = torch.nn.Linear(512, 10)

In [ ]:
class PointNetFineTuned(torch.nn.Module):
    def __init__(self,trained_model, ):
        super().__init__()
        # Feature extraction
        self.pre_trained_model = trained_model

    def forward(self, data):
            out = self.pre_trained_model(data, train=False)
            return F.log_softmax(out, dim=1)

In [ ]:
pointnet_finetuned= PointNetFineTuned(pointnet2_saved).to(device)
optimizer = torch.optim.Adam(pointnet_finetuned.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
for epoch in range(0, 100):
    loss = train_model(pointnet_finetuned,train_loader)
    test_loss,test_acc = evaluate_model(pointnet_finetuned,val_loader)
    print(f'Epoch {epoch:03d}, Train_Loss: {loss:.4f},Test_Loss: {test_loss:.4f}, Test_acc: {test_acc:.4f}')
    scheduler.step()


In [ ]:
test_model(pointnet_finetuned,test_loader)

In [ ]:
del pointnet_finetuned

**2. Finetune Dynamic graph CNN model**

In [ ]:
edgeconv_path='/kaggle/input/trained-models/EdgeConvNet_Contrasrive_saved_model_final.pth'
EdgeConvNet_saved=EdgeConvNet_ContrasriveNet().to(device)
EdgeConvNet_saved.load_state_dict(torch.load(edgeconv_path))
EdgeConvNet_saved.ph3 = torch.nn.Linear(512, 10)

In [ ]:
class EdgeConvFineTuned(torch.nn.Module):
    def __init__(self,trained_model, ):
        super().__init__()
        # Feature extraction
        self.pre_trained_model = trained_model

    def forward(self, data):
            out = self.pre_trained_model(data, train=False)
            return F.log_softmax(out, dim=1)

In [ ]:
EdgeConvNet_finetuned=EdgeConvFineTuned(EdgeConvNet_saved).to(device)
optimizer = torch.optim.Adam(EdgeConvNet_finetuned.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
for epoch in range(0, 100):
    loss = train_model(EdgeConvNet_finetuned,train_loader)
    test_loss,test_acc = evaluate_model(EdgeConvNet_finetuned,val_loader)
    print(f'Epoch {epoch:03d}, Train_Loss: {loss:.4f},Test_Loss: {test_loss:.4f}, Test_acc: {test_acc:.4f}')

In [ ]:
test_model(EdgeConvNet_finetuned,test_loader)

In [ ]:
#garbage collection
gc.collect()
torch.cuda.empty_cache()